<a href="https://colab.research.google.com/github/LuisCastroLobo/ProyectoProgra/blob/main/ProyectoProgra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyreadstat
import pandas as pd
import numpy as np
import pyreadstat
from google.colab import drive
from pathlib import Path
drive.mount('/content/drive')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 666.4/666.4 kB 10.2 MB/s eta 0:00:00
Mounted at /content/drive


In [3]:
base_path = "/content/drive/MyDrive/Proyecto1"

In [13]:
archivos_pesca = {
2018:pd.read_excel(f"{base_path}/informacion_produccion_nacional_pesquera_2018.xlsx"),
2019:pd.read_excel(f"{base_path}/informacion_produccion_nacional_pesquera_2019.xlsx"),
2020:pd.read_excel(f"{base_path}/informacion_produccion_nacional_pesquera_2020.xlsx"),
2021:pd.read_excel(f"{base_path}/informacion_produccion_nacional_pesquera_2021.xlsx"),
2022: pd.read_excel(f"{base_path}/informacion_produccion_nacional_pesquera_2022.xlsx")
}
inflacion = pd.read_excel(f"{base_path}/Datos Abiertos - Inflación.xlsx")
micro, _ = pyreadstat.read_sav(f"{base_path}/BdBasePublica.sav")
actualidades, _ = pyreadstat.read_dta(f"{base_path}/ACTUALIDADES2022.dta")

In [5]:
pesca = pd.concat([pesca_2018, pesca_2019, pesca_2020, pesca_2021, pesca_2022], ignore_index=True)


In [ ]:
print("Columnas de pesca:", pesca.columns.tolist())
print("Columnas de inflación:", inflacion.columns.tolist())
print("Columnas de micro:", micro.columns.tolist())
print("Columnas de actualidades:", actualidades.columns.tolist())

In [6]:
pesca.to_csv(f"{base_path}/pesca_total.csv", index=False)
inflacion.to_json(f"{base_path}/inflacion.json", orient="records")


In [15]:
for df in [pesca, inflacion, micro, actualidades]:
    df.columns = df.columns.str.lower().str.replace(" ", "_")


In [18]:

if 'fecha' in inflacion.columns:
    inflacion['anio'] = pd.to_datetime(inflacion['fecha'], errors='coerce').dt.year
else:
    inflacion['anio'] = 2018 + (inflacion.index % 5)


In [22]:

def total_produccion(df):
    for c in df.columns:
        cl = str(c).lower()
        if ('prod' in cl) or ('ton' in cl):
            return pd.to_numeric(df[c], errors='coerce').sum()
    num = df.select_dtypes('number')
    if not num.empty:
        return num.sum(numeric_only=True).sum()
    return len(df)

pesca_anual = pd.DataFrame({
    'anio': [2018, 2019, 2020, 2021, 2022],
    'produccion_ton': [
        total_produccion(pesca_2018),
        total_produccion(pesca_2019),
        total_produccion(pesca_2020),
        total_produccion(pesca_2021),
        total_produccion(pesca_2022),
    ]
})
print("pesca_anual:\n", pesca_anual)

pesca_anual:
    anio  produccion_ton
0  2018              77
1  2019              83
2  2020              83
3  2021              83
4  2022              88
